## Colab Stuff

It's highly recommended to download the dataset first to your drive and work on Google Colab to save your time (and your local machine too:)))

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd '/content/gdrive/My Drive/RL'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/RL


## Datasets

The official website of `PASCAL VOC` dataset is not working. So, you will need to search for a new way:
1. This is an alternative website which we used: [https://deepai.org/dataset/pascal-voc](https://deepai.org/dataset/pascal-voc)
2. Use our shared version from [Google Drive](https://drive.google.com/drive/folders/1wyB3XcD8EhiZI1BZYZ5SawG-DOwZzBV_?usp=sharing) in case the alternative website isn't available too.

P.S. Our drive folder contains only `PASCAL VOC 2007`. If you need a compressed folder for `PASCAL VOC 2012`, you can easily find it [here](https://drive.google.com/file/d/1XU7qwUVnV2S_iiWfb4DEmoxjbnQJxpHE/view?usp=sharing://) on our Google Drive too :)

## Reading Dataset

[Errno 2] No such file or directory: 'DRL-Object-Detection-main/'
/home/boweiche/Documents/robotics/DRL-Object-Detection


In [1]:
from utils.agent import *
from utils.dataset import read_voc_dataset
from IPython.display import clear_output
import tqdm.notebook as tq
import pickle
%reload_ext autoreload
%autoreload 2

train_loader2007, val_loader2007 = read_voc_dataset(path="./data/PascalVOC2012" ,year='2012', download=True)

  0%|          | 0/1999639040 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Dataset Organization

In [6]:
#put datalist in list in case of combination of different datasets
#For example: [train_loader2007, train_loader2012]
datasets_per_class_train = sort_class_extract([train_loader2007])

  0%|          | 0/5717 [00:00<?, ?it/s]

In [7]:
datasets_per_class_test = sort_class_extract([val_loader2007])

  0%|          | 0/5823 [00:00<?, ?it/s]

## Training Loop

In [10]:

for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Classe "+str(classe)+"...")
    #agent = Agent(classe, alpha=0.2, num_episodes=15, load=False, model_name='vgg16')
    agent = Agent_3alpha(classe, alpha=0.2, num_episodes=15, load=False, model_name='vgg16_3step')
    agent.train(datasets_per_class_train[classe])
    del agent
    torch.cuda.empty_cache()


  0%|          | 0/1 [00:00<?, ?it/s]

Classe cat...
Episode 0
Saved
Complete
Episode 1
Saved
Complete
Episode 2
Saved
Complete
Episode 3
Saved
Complete
Episode 4
Saved
Complete
Episode 5
Saved
Complete
Episode 6
Saved
Complete
Episode 7
Saved
Complete
Episode 8
Saved
Complete
Episode 9
Saved
Complete
Episode 10
Saved
Complete
Episode 11
Saved
Complete
Episode 12
Saved
Complete
Episode 13
Saved
Complete
Episode 14
Saved
Complete


**TESTING**

In [16]:
torch.cuda.empty_cache()
results = {}
for i in classes:
    results[i] = []
model_name='vgg16_3step'
for i in tq.tqdm(range(len(classes))):
    classe = classes[i]
    print("Class "+str(classe)+"...")
    agent = Agent_3alpha(classe, load=True, model_name=model_name)
    res = agent.evaluate(datasets_per_class_test[classe])
    results[classe] = res

  0%|          | 0/1 [00:00<?, ?it/s]

Class cat...
Predicting boxes...


/home/boweiche/Documents/robotics/DRL-Object-Detection/utils/agent.py:714: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  if (action-1)//8 == 0: alpha = self.alpha * 0.5
/home/boweiche/Documents/robotics/DRL-Object-Detection/utils/agent.py:715: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  elif (action-1)//8 == 1: alpha = self.alpha * 

Computing recall and ap...
Final result : 
              0.4        0.5        0.6
ap      65.533981  54.530744  42.556634
recall  65.533981  54.530744  42.556634


In [17]:
file_name = 'classes_results_' + model_name + '.pickle'
with open(file_name, 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

**VISUALIZE**

In [19]:
from utils.tools import classes
classe = random.choice(classes)
indices = np.random.choice(list(datasets_per_class_test[classe].keys()), size=5, replace=False)
agent = Agent_3alpha(classe, load=True, model_name=model_name)

print("Class: " + classe)
for index in indices:
    image, gt_boxes = extract(index, datasets_per_class_test[classe])
    agent.predict_image(image, plot=True)

Class: cat


/home/boweiche/Documents/robotics/DRL-Object-Detection/utils/agent.py:714: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  if (action-1)//8 == 0: alpha = self.alpha * 0.5
/home/boweiche/Documents/robotics/DRL-Object-Detection/utils/agent.py:715: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  elif (action-1)//8 == 1: alpha = self.alpha * 